<h2 style="text-align:center; color:#4CAF50">Group Partners:</h2>
<ul style="list-style-type:none; padding-left: 0; text-align:center;">
  <li style="margin-bottom: 10px; font-size: 18px;">Muhammad Hamza _ 22F-3134</li>
  <li style="margin-bottom: 10px; font-size: 18px;">Abdul Hanan _ 22F-3104</li>
</ul>


In [16]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import sys
import heapq
from lib.EightPuzzle import EightPuzzle

In [17]:
root = Tk()

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

### Two must functions: 
* This will be used for reconstructing changes in the already given state(list)

In [18]:
def get_next_states(state):
    next_states = []
    zero_index = state.index(0)
    row, col = zero_index // 3, zero_index % 3
    
    actions = {'UP': (-1, 0), 'LEFT': (0, -1), 'DOWN': (1, 0), 'RIGHT': (0, 1)} #defining actions for movement
    
    for action, (direction_row, direction_col) in actions.items():   # action dictionary contains, matrix distance and directions
        next_row, next_col = row + direction_row, col + direction_col  #for getting positions of next rows and columns
        if 0 <= next_row < 3 and 0 <= next_col < 3:   #check for if the action might result in a valid next position
            next_state = list(state)   #impossible for modification, hence converted to list
            next_zero_index = next_row * 3 + next_col
            next_state[zero_index], next_state[next_zero_index] = next_state[next_zero_index], 0  #swapping 0 with adjacent tile, as at a null space we have 0
            next_states.append((tuple(next_state), action))  #back to tuple
            
    return next_states

* This will be used for getting next shape of state(list)

In [19]:
def reconstruct_path(from_, current_state):
    path = []   #list for storing actions to current state
    while current_state is not None:    #until we reach the initial state
        if current_state in from_:   #check for info about parent of the current state
            parent_state, action = from_[current_state]
            path.append(action)   #retrieval and adding the action to our path
            current_state = parent_state    #now finally moving to the parent tile and performing the action again
        else:   #other wise, if there is no information about the parent state
            break    #just stop checking
            
    path = path[::-1]    #returning the reverse path
    return path

# Greedy Best First Search:

## 1. Manhattan's distance

In [11]:
def manhattan_distance(state):
    total_distance = 0
    for i in range(3):
        for j in range(3):
            value = state[(3 * i) + j]   #for getting value at every position
            if value != 0:
                current_row = (value - 1) // 3   #will get the position of row by dividing it with absolute 3
                current_col = (value - 1) % 3    #will get the position of col by taking remainder after dividing it by 3
                
                row_diff = i - current_row  #for checking the difference between current row and target row
                col_diff = j - current_col  #                same with the columns
                
                if row_diff < 0:    #if the answer goes negative, sometimes, will take mod '||'
                    row_diff *= -1
                if col_diff < 0:
                    col_diff *= -1
                
                total_distance += row_diff + col_diff
    return total_distance

def solve():
    MAX_SCORE = 81  #defining a large integer value for proper checking
    open_set = [(manhattan_distance(tuple(state)), tuple(state))]  #using priority queue
    heapq.heapify(open_set)   #made open_set a min-heap to make my task easier
    came_from = {}
    g_score = {tuple(state): 0}  # Convert to tuple
    f_score = {tuple(state): manhattan_distance(tuple(state))}  #converting to tuple

    while open_set:
        temp, current_state = heapq.heappop(open_set)   #deleting value from the last and storing in current state
        if current_state == tuple(range(9)):  #converting to tuple
            return reconstruct_path(came_from, current_state)
        
        for neighbor_state, action in get_next_states(list(current_state)):  #converting to list for modification
            tentative_g_score = g_score.get(current_state, MAX_SCORE) + 1
            if tentative_g_score < g_score.get(neighbor_state, MAX_SCORE):
                came_from[neighbor_state] = (current_state, action)
                g_score[neighbor_state] = tentative_g_score
                f_score[neighbor_state] = tentative_g_score + manhattan_distance(neighbor_state)
                heapq.heappush(open_set, (f_score[neighbor_state], neighbor_state))

    return []  # If no solution found


## 2. Misplaced Tiles

In [20]:
def misplaced_tiles(state):
    misplaced = 0
    for i in range(9):
        if state[i] != i:
            misplaced += 1
    return misplaced

def solve():
    MAX_score = 81
    open_set = [(misplaced_tiles(tuple(state)), 0, tuple(state))]  #using the priority queue
    came_from = {}
    g_score = {tuple(state): 0}  #converting to tuple
    
    while open_set:
        temp, current_cost, current_state = min(open_set)  #getting the path with minimum path cost
        open_set.remove((misplaced_tiles(current_state), current_cost, current_state))  #removing the item from the queue
        if current_state == tuple(range(9)):  #converting to tuple
            return reconstruct_path(came_from, current_state)
        
        for neighbor_state, action in get_next_states(list(current_state)):  #converting to list for modification
            tentative_g_score = current_cost + 1
            if tentative_g_score < g_score.get(neighbor_state, MAX_score):
                came_from[neighbor_state] = (current_state, action)
                g_score[neighbor_state] = tentative_g_score
                open_set.append((misplaced_tiles(neighbor_state), tentative_g_score, neighbor_state))  # Add the new item to the queue
    
    return []  # If no solution found


## A* search

In [60]:
def manhattan_distance(state):
    total_distance = 0
    for i in range(3):
        for j in range(3):
            value = state[(3 * i) + j]   #for getting value at every position
            if value != 0:
                current_row = (value - 1) // 3   #will get the position of row by dividing it with absolute 3
                current_col = (value - 1) % 3    #will get the position of col by taking remainder after dividing it by 3
                
                row_diff = i - current_row  #for checking the difference between current row and target row
                col_diff = j - current_col  #                same with the columns
                
                if row_diff < 0:    #if the answer goes negative, sometimes, will take mod '||'
                    row_diff *= -1
                if col_diff < 0:
                    col_diff *= -1
                
                total_distance += row_diff + col_diff
    return total_distance

def solve(state):
    MAX_score = 81
    open_set = [(manhattan_distance(tuple(state)), tuple(state))]  # Use priority queue
    came_from = {}
    g_score = {tuple(state): 0}  # Convert to tuple
    f_score = {tuple(state): manhattan_distance(tuple(state))}  # Convert to tuple
    
    while open_set:
        current_priority, current_state = min(open_set)  # Get the item with minimum priority
        open_set.remove((current_priority, current_state))  # Remove the item from the queue
        if current_state == tuple(range(9)):  # Convert to tuple
            return reconstruct_path(came_from, current_state)
        
        for neighbor_state, action in get_next_states(list(current_state)):  # Convert to list for modification
            cost = 0
            if action == 'UP':
                cost = 2
            elif action == 'LEFT':
                cost = 1
            elif action == 'RIGHT':
                cost = 3
            elif action == 'DOWN':
                cost = 4

            tentative_g_score = g_score.get(current_state, MAX_score) + cost
            if tentative_g_score < g_score.get(neighbor_state, MAX_score):
                came_from[neighbor_state] = (current_state, action)
                g_score[neighbor_state] = tentative_g_score
                f_score[neighbor_state] = tentative_g_score + manhattan_distance(neighbor_state)
                open_set.append((f_score[neighbor_state], neighbor_state))  # Add the new item to the queue
    
    return []  # If no solution found


In [21]:
def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = solve()
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.50))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))

def exit():    #made this one myself, learnt from documentation
    root.destroy()
        
def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender',command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'), fg='black'
                  , bg='lavender', command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init), bg='blue',fg='white')
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Exit', font=('Helvetica', 30, 'bold'), width=8, command=partial(exit), bg='blue',fg='white')
    solve_btn.grid(row=3, column=1, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps), bg='blue',fg='white')
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


['UP', 'RIGHT', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP']
['DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP']
